#### Notebook for QOSF Cohort 9 - Screening Task
Name: Hari Priya Narahari

#### Task 1 less than k

Given a positive integer “k” and a list of integer numbers, look for the numbers within the list, that are less than k. Consider an appropriate number of qubits and explain why your proposal is valid for all kinds of numbers in case 

```
def less_than_k (int:k, list[int] ,list_n):

    k : integer value that is the positive number to compare in list_n,
    list_n : integer list that has positive numbers.
    Return the numbers that are in list_n and are less than k 

```

Example:

A = less_than_k (7,[4,9,11,14,1,13,6,15])
print(A)

“4,1,6”


##### Solution:

Paper reference: https://www.researchgate.net/publication/228574906_Quantum_bit_string_comparator_Circuits_and_applications

Comparator Reference: https://pyqml.medium.com/the-quantum-bit-comparator-463911f7bcd3



In [2]:
from qiskit import QuantumCircuit, Aer, transpile, assemble, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram
from qiskit import execute, Aer
import numpy as np

def int_to_binary_string(num, num_bits):
    """Converts an integer to a binary string with specified number of bits."""
    binary_str = bin(num)[2:]  # Convert to binary string and remove '0b' prefix
    padded_str = binary_str.zfill(num_bits)  # Pad with leading zeros if necessary
    return padded_str

def encode(bit):
    """
    Constructs a quantum circuit to encode a single bit into a quantum state. 
    It takes a single argument bit, which is the bit to be encoded. 
    If the bit is '1', it applies an X gate to flip the state of the qubit.
    """
    qr = QuantumRegister(1, "number")
    qc = QuantumCircuit(qr)
    if (bit == "1"):
        qc.x(qr[0])
    return qc

def bit_compare():
    """
    Constructs a quantum circuit to compare two bits represented by qubits. 
    It consists of two quantum registers, qr with 2 qubits for the bits and aux with 2 qubits as auxiliary qubits. 
    """
    qr = QuantumRegister(2, "bits")
    aux = QuantumRegister(2, "aux")
    
    qc = QuantumCircuit(qr, aux)
    qc.x(qr[1])
    qc.mcx(qr, aux[0])
    qc.x(qr[0])
    qc.x(qr[1])
    qc.mcx(qr, aux[1])
    qc.x(qr[0])
    
    return qc

def compare_bitstring(b, a,  num_bits):
    """
    Compares two bit strings using a quantum circuit. 
    It takes bitstring_b and bitstring_a as input along with the number of bits num_bits.
    It constructs a quantum circuit to compare the bit strings using the encode and bit_compare functions. 
    It measures the output qubits and returns the counts of measurement outcomes. 
    """
    bits = num_bits
    qa = QuantumRegister(bits, "a")
    qb = QuantumRegister(bits, "b")
    qraux = QuantumRegister(2*bits, "aux")
    qrint = QuantumRegister(bits-1, "int")
    cr = ClassicalRegister(2)

    qc = QuantumCircuit(qa, qb, qraux, qrint, cr)

    for i in range(bits):
        qc.append(encode(a[i]), [qa[i]])
        qc.append(encode(b[i]), [qb[i]])
        qc.append(bit_compare(), [qa[i], qb[i], qraux[2*i], qraux[2*i+1]])
        
        if i < bits-1:
            qc.x(qraux[2*i])
            qc.x(qraux[2*i+1])
            qc.mcx([qraux[2*i], qraux[2*i+1]], qrint[i])
            qc.x(qraux[2*i])
            qc.x(qraux[2*i+1])
        
    for i in range(0, bits-1):
        qc.mcx([qraux[2*(-i-1)],  qrint[-i]], qraux[2*(-i)])
        qc.mcx([qraux[2*(-i-1)+1],  qrint[-i]], qraux[2*(-i)+1])
        
    qc.measure(qraux[0], cr[0])
    qc.measure(qraux[1], cr[1])
    
        # Tell Qiskit how to simulate our circuit
    backend = Aer.get_backend('qasm_simulator') 

    # Do the simulation, returning the result
    result = execute(qc,backend, shots=1000).result()

    # get the probability distribution
    counts = result.get_counts()
    #print(counts)

    if '01' in counts:
        return 1
    else:
        return 0



def less_than_k(reference_val,values):
    """
    Compares a list of values with a reference value using the compare_bitstring function. 
    It takes reference_val: the reference value, and values: a list of values to be compared. 
    It iterates over each value, determines the number of bits required for comparison, converts the value and the reference value to binary strings, 
    and compares them using the compare_bitstring function. It returns a list results containing the comparison results for each value.

    The number of bits have been calculated keeping in mind how the circuit works, this has been a bit of a novel step implemented on the existing solution.
    """
    results = []
    for val in values:
        num_bits = int(np.ceil(max([np.log2([val+1]),np.log2([reference_val]),2])))
        #print(num_bits)
        val = int_to_binary_string(val, num_bits)
        reference_value = int_to_binary_string(reference_val, num_bits)
        #print(val,reference_value,num_bits)
        results.append(compare_bitstring(val,reference_value,num_bits))
    return results
    
results = []
values = [1,2,3,4,5,7,8,9,10,11]
reference_val = 20

results = less_than_k(reference_val,values)
print("Values less than",reference_val,":")
for i in range(len(results)):
    if results[i] == 1:
        print(values[i],end=" ")


Values less than 20 :
1 2 3 4 5 7 8 9 10 11 

#### Improvements needed:

1. The solution fails when values with a intermediate 0 is compared. Eg: Solution fails for 10 when comapred to 8 as 10: 1010, 8:1000
2. The solution does not wrok for negative numbers.